In [ ]:
# This takes photometry of stars through apertures arranged on a grid which moves
# to compensate for poor telescope guiding

# Created 2021 Aug. 20 by E.S.

In [35]:
import pandas as pd
import numpy as np
#from astropy.stats import sigma_clipped_stats
#from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import glob
import os
from numpy.linalg import inv

%matplotlib qt

In [2]:
stem_data = "/Users/bandari/Documents/git.repos/kriz_projects/data/2021-08-04/00_dark_subtracted_flat_fielded/"
stem_star_positions = "/Users/bandari/Documents/git.repos/kriz_projects/notebooks_for_development/output_other/"

In [3]:
# names of files

file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

file_names_pos = sorted(glob.glob(stem_star_positions + "V1432_Aql-*dat"))

# file of measured displacements between images
file_name_disp = "displacements_v1432_aql.csv"

In [51]:
# machinery to find rotation and translation with SVD:

# Matrix of empirically found positions        F: [ [x_1' y_1' z_1'=0], [x_2' y_2' z_2'=0], [x_3' y_3' z_3'=0] ]
# Rotation matrix                              R
# Matrix of estimated, translated positions    T: [ [x_1 y_1 z_1=0], [x_2 y_2 z_2=0], [x_3 y_3 z_3=0] ]

# fake data
T_estimate = np.array([[0.5, 0.5*np.sqrt(3)/2., 0.], [-0.5*np.sqrt(3)/2., 0.5, 0.], [-0.5, -0.5*np.sqrt(3)/2.,0.]])
F = np.array([[0, 1., 0],[-1, 0, 0],[0, -1, 0]])

import svdt
R, L, RMSE = svdt.svd(A=T_estimate, B=F)

print(R)
print(L)
print(RMSE)

In [34]:
# read in image displacements
disp_images = pd.read_csv(stem_star_positions+file_name_disp)
# read in first image's star locations (we will shift this)
initial_star_pos = pd.read_csv(file_names_pos[0], delim_whitespace=True)

## ## START FOR-LOOP

for num_star_file in range(2200,2201): #1200,2200 len(file_names_stars)):

    # read in star image
    star_image = fits.open(file_names_stars[num_star_file])[0].data

    # retrieve displacement data
    idx = disp_images["file_name"]==str(os.path.basename(file_names_stars[num_star_file]))
    x_disp = disp_images["x_off_pix"].where(idx).dropna().values[0]
    y_disp = disp_images["y_off_pix"].where(idx).dropna().values[0]

    # shift grid of first (i.e., [0]) array of images by this much; this is a first-order correction
    # (n.b. we use this grid and translate it in order to preserve the ID information of the stars;
    # otherwise, if we re-find the stars in each frame we don't know which is which)
    positions_transl = np.transpose((np.subtract(initial_star_pos['xcentroid'],x_disp), 
                              np.subtract(initial_star_pos['ycentroid'],y_disp)))
    
    ## ## fcn to find the positions of the three brightest stars closest to their estimated positions, 
    ## ## and with a brightness that is consistent (brightest stars have IDs 26, 272, 99 [I believe; check])
    
    ## ## insert SVD functionality here to find residual translation and rotation
    
    ## ## make the residual transformation to the aperture locations

    
    

    '''
    print(positions)
    
    apertures = CircularAperture(positions, r=4.)
    

    phot_table = aperture_photometry(star_image, apertures)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output
    
    
    ## ## START TEST
    # convert to DataFrame
    df_phot = phot_table.to_pandas()
    
    # remove rows with coordinates that put the stars outside the 512x512 frame, or close to edge
    edge_buffer = 15 # pix
    #phot_table = phot_table[~mask]
    idx_edge_xs = np.logical_or(df_phot["xcenter"] > np.subtract(512,edge_buffer), df_phot["xcenter"] < edge_buffer)
    idx_edge_ys = np.logical_or(df_phot["ycenter"] > np.subtract(512,edge_buffer), df_phot["ycenter"] < edge_buffer)
    df_phot_drop_edges = df_phot.drop(df[np.logical_or(idx_edge_xs,idx_edge_ys)].index, inplace = False)
    
    # sort: brightest stars first
    df_phot_drop_edges = df_phot_drop_edges.sort_values("aperture_sum", axis=0, ascending=False)
    
    print(df_phot)
    print(df_phot_drop_edges)
    
    ## ## END TEST
    '''
    # write text file with aperture photometry
    '''
    text_file_name = "output_other/aper_photom_"+str(os.path.basename(file_names_stars[num_star_file])).split(".")[-2]+".dat"
    ascii.write(phot_table, text_file_name, overwrite=False)
    print("Wrote out "+text_file_name)

    # write FYI plot
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    '''


## ## END FOR-LOOP

[[ 2.65534650e+02  1.56149222e+01]
 [-8.45250549e+01  1.89056526e+01]
 [ 2.74566330e+02  1.77664479e+01]
 [ 3.97172260e+02  1.88949629e+01]
 [-7.63874965e+01  2.38088388e+01]
 [-6.79796940e+01  2.39684733e+01]
 [ 7.75203700e+01  2.33791306e+01]
 [ 3.03137300e+02  2.43727916e+01]
 [-7.21118500e+00  2.54734770e+01]
 [-5.97740890e+01  2.70837900e+01]
 [ 3.76691050e+02  2.75297400e+01]
 [-7.77209860e+01  2.87043420e+01]
 [ 3.02286960e+02  3.24907890e+01]
 [-5.43440550e+01  3.93314890e+01]
 [ 1.81342050e+02  4.26154490e+01]
 [ 3.22564500e+02  4.57582500e+01]
 [ 3.93587500e+02  4.51167560e+01]
 [ 4.22238610e+02  4.57978270e+01]
 [ 9.48971200e+01  4.86567010e+01]
 [ 8.19611300e+01  5.28930000e+01]
 [ 1.54863050e+02  5.39822950e+01]
 [ 1.74267330e+02  5.47131780e+01]
 [ 3.87271230e+02  5.41991660e+01]
 [ 3.15018980e+02  5.56768220e+01]
 [ 2.18108970e+02  5.73363940e+01]
 [ 2.92786930e+02  5.67512400e+01]
 [ 3.58494870e+02  5.62894660e+01]
 [ 3.46542000e+01  5.72925450e+01]
 [-7.84930159e+01  5

/Users/bandari/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [10]:
from numpy.linalg import inv
a = np.array([[1., 2.], [3., 4.]])
ainv = inv(a)

In [12]:
print(a)

[[1. 2.]
 [3. 4.]]


In [11]:
np.allclose(np.dot(a, ainv), np.eye(2))
np.allclose(np.dot(ainv, a), np.eye(2))

True

In [42]:
'''
# for plotting
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)
'''

In [44]:
# star images
file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

# darks
file_names_darks = sorted(glob.glob(stem_data + "d*fit"))

# flats
file_names_flats = sorted(glob.glob(stem_data + "flat*fit"))